In [1]:
import os
import re
import time
import pickle
import asyncio
import nest_asyncio
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from IPython.display import clear_output

from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import ClovaXEmbeddings
from langchain_community.chat_models import ChatClovaX
from pymilvus import connections, utility
from langchain_community.vectorstores.milvus import Milvus
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 환경 설정 및 Milvus/임베딩 초기화

load_dotenv(dotenv_path=r"C:\Kill_the_RAG\Project\Aiffel_final_project\.env")

os.environ["NCP_CLOVASTUDIO_API_KEY"] = os.getenv("NCP_CLOVASTUDIO_API_KEY")
os.environ["NCP_CLOVASTUDIO_API_URL"] = os.getenv(
    "NCP_CLOVASTUDIO_API_URL", "https://clovastudio.stream.ntruss.com/"
)

connections.connect(
    alias="default",
    host=os.getenv("MILVUS_HOST", "localhost"),
    port=os.getenv("MILVUS_PORT", "19530"),
)

ncp_embeddings = ClovaXEmbeddings(model="bge-m3")
llm_clova = ChatClovaX(model="HCX-003", max_tokens=2048)

In [3]:
# 임베딩 파일 및 문서 구성

embedding_file = r"C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\semifinal_embedding\embedding_semifinal.pkl"
if os.path.exists(embedding_file):
    with open(embedding_file, "rb") as f:
        saved_data = pickle.load(f)
    all_text_embedding_pairs = saved_data["embeddings"]
    all_metadata_list = saved_data["metadata"]
    print("임베딩 데이터 불러오기")
else:
    raise FileNotFoundError(f"임베딩 파일을 찾을 수 없습니다: {embedding_file}")

metadata_mapping = {
    "ISBN": "ISBN",
    "페이지": "page",
    "가격": "price",
    "제목": "title",
    "저자": "author",
    "분류": "category",
    "저자소개": "author_intro",
    "책소개": "book_intro",
    "목차": "table_of_contents",
    "출판사리뷰": "publisher_review",
    "추천사": "recommendation",
}

all_metadata_list_mapped = []
for meta in all_metadata_list:
    mapped_meta = {metadata_mapping.get(key, key): value for key, value in meta.items()}
    all_metadata_list_mapped.append(mapped_meta)

documents = [
    Document(page_content=pair[0], metadata=meta)
    for pair, meta in zip(all_text_embedding_pairs, all_metadata_list_mapped)
]

임베딩 데이터 불러오기


In [4]:
# Milvus 벡터 DB 생성
collection_name = "book_rag_db"
if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)

vectorstore = Milvus(
    embedding_function=ncp_embeddings,
    collection_name=collection_name,
    connection_args={"host": "localhost", "port": "19530"},
    auto_id=True,
)

texts = [pair[0] for pair in all_text_embedding_pairs]
embeds = [pair[1] for pair in all_text_embedding_pairs]


def precomputed_embed_documents(cls, input_texts):
    if input_texts != texts:
        raise ValueError(
            "ERROR : 입력 텍스트 순서가 사전 계산된 임베딩과 일치하지 않음"
        )
    return embeds


ClovaXEmbeddings.embed_documents = classmethod(precomputed_embed_documents)
vectorstore.add_texts(
    texts=texts, metadatas=all_metadata_list_mapped, embeddings=embeds
)

dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
dpr_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_clova, retriever=dense_retriever, return_source_documents=True
)

C:\Users\computer\AppData\Local\Temp\ipykernel_25152\1961100548.py:6: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectorstore = Milvus(


In [ ]:
# 유틸


def extract_field(text, field_name):

    pattern = rf"{re.escape(field_name)}\s*:\s*(.*)"
    match = re.search(pattern, text)
    return match.group(1).strip() if match else ""


MIN_INFO_LENGTH = 10

In [18]:
# 공용 & 페르소나별 프롬프트 템플릿

decision_prompt_template = PromptTemplate(
    template="""
[대화 맥락]
사용자 대화 내역:
{{ history }}
사용자의 최신 질문: "{{ query }}"

[역할 및 목표]
{{ role_instructions }}
현재 대화 상황과 질문의 맥락을 분석하여 아래 중 하나의 행동을 출력해라:
- "추천": 사용자가 책 추천을 명확히 요청한 경우, 추가 정보 없이 바로 추천을 진행. & 추가 질문에서 책 추천을 원한다는 답변에 긍정적으로 응답한 경우, 바로 추천을 진행.
- "추가 질문": 책 추천을 위해 더 세부적인 선호도 정보(예: 작가, 시대, 장르 등)가 필요하면, 구체적인 질문을 던져서 정보를 요청. & 충분한 사용자 정보를 수집했다고 판단할 경우, 추천을 원하냐고 질의.
- 질문에 대해서 사용자가 "모호한", "부정적인" 답변을 할 경우 넘어가라.
- "~~비슷한" 이라고 사용자가 답변할 경우, 쿼리에 그대로 반영하는것이 아닌 너의 지식정보를 활용해서 특징을 쿼리로 반영하라.
    - 예시 : 히가시노 게이고 작가랑 비슷한 -> 미스터리, 추리 소설

[최종 출력 형식 - 반드시 아래 내용만 출력]
{% raw %}
행동: "{{행동 (추천/추가 질문}}"
추천 책 정보: "{{책 제목 및 상세 정보, 정보 충분 시; 정보 부족 시 빈 문자열}}"
추가 질문: "{{추가 질문, 정보 보완 필요 시; 충분하면 빈 문자열}}"
{% endraw %}
""",
    input_variables=["history", "query", "role_instructions"],
    template_format="jinja2",
)

final_query_generation_template = PromptTemplate(
    template=""""
[대화 요약]
{{ history }}

[사용자 요청]
{{ query }}

[페르소나 정보]
{{ persona_info }}

[사용자 선호도]
{{ preferences }}

위 정보를 바탕으로, 책 추천에 활용 가능한 핵심 선호도 정보(예: 장르, 주제, 분위기 등)를 반영하여 자연스러운 한 문장의 "최종 검색 쿼리"를 작성해라.

- "최종 검색 쿼리"는 반드시 "추천 해줘"로 끝나도록 한다.
- "최종 검색 쿼리"를 질문 형태로 작성하지 않는다.
- '추가 질문'이라는 표현은 "최종 검색 쿼리"에 절대 사용하지 않는다.

최종 출력 형식:
쿼리: <최종 검색 쿼리>
""",
    input_variables=["history", "query", "persona_info", "preferences"],
    template_format="jinja2",
)

# 페르소나 프롬프트

literature_role = "너는 감성적이고 문학적인 도서 추천 챗봇이다. 사용자의 감정과 취향을 섬세하게 파악하여, 감성적인 언어로 책을 추천해라."
science_role = "너는 정확하고 논리적인 과학/기술 도서 추천 챗봇이다. 사용자의 관심 분야와 요구를 분석하여, 명확하고 구체적인 정보로 책을 추천해라."
general_role = "너는 친절하고 신뢰할 수 있는 범용 도서 추천 챗봇이다.사용자의 관심사와 목적을 파악하여, 다양한 분야의 책을 적절하게 추천해라. 문학, 과학 외에도 자기계발, 역사, 에세이, 경제경영 등 모든 장르에 유연하게 대응해라."

In [ ]:
# 질문 임베딩 기록용 리스트
previous_question_embeddings = []


def is_similar_question(new_emb, prev_embeds, threshold=0.88):
    if not prev_embeds:
        return False
    sim_scores = cosine_similarity([new_emb], prev_embeds)[0]
    max_score = max(sim_scores)
    print(f"[중복 유사도 판단] Max = {max_score:.3f}")
    return max_score > threshold

In [ ]:
# 비동기 체인 구성 & 디버깅 로깅 함수


async def async_invoke(chain: LLMChain, vars_dict: dict, step_name: str) -> dict:
    """
    체인 호출을 비동기로 수행하고, 단계별 디버깅 출력과 예외 처리를 수행합니다.
    """
    try:

        print(f"\n[디버그] {step_name} 호출 전 변수: {vars_dict}")

        # 동기 함수인 llm.invoke를 비동기로 실행 (to_thread 사용)
        result = await asyncio.to_thread(chain.invoke, vars_dict)
        print(f"[디버그] {step_name} 결과: {result}")

        return result

    except Exception as e:

        print(f"[에러] {step_name}에서 예외 발생: {str(e)}")

        return {"text": ""}


async def async_invoke_llm(prompt: str, step_name: str) -> str:
    """
    단일 프롬프트 문자열을 llm.invoke로 비동기 호출하여 결과를 반환합니다.
    """
    try:

        print(f"\n[디버그] {step_name} 프롬프트 호출:\n{prompt}")
        response = await asyncio.to_thread(llm_clova.invoke, prompt)
        result_text = response.text().strip()
        print(f"[디버그] {step_name} 응답: {result_text}")

        return result_text

    except Exception as e:

        print(f"[에러] {step_name}에서 예외 발생: {str(e)}")

        return ""

In [ ]:
# BaseRAGPipeline 클래스 (체인 구성 추가 + 디버깅 방식 변경 + 비동기 방식)


class BaseRAGPipeline:

    def __init__(self, config, llm, retriever, qa_chain, documents, embedding_model):

        self.config = config

        self.llm = llm

        self.retriever = retriever

        self.qa_chain = qa_chain

        self.documents = documents

        self.embedding_model = embedding_model

        self.query_history = []

        self.user_preferences = defaultdict(list)

        self.preferences_text = ""

        self.decision_chain = LLMChain(llm=self.llm, prompt=decision_prompt_template)

        self.final_query_generation_chain = LLMChain(
            llm=self.llm, prompt=final_query_generation_template
        )

    async def summarize_user_preferences(self, existing_preferences, new_input):

        prompt = (
            f"다음 사용자 선호도 내용들을 하나의 자연스러운 문장으로 요약해라:\n"
            f"기존 선호도: {existing_preferences}\n"
            f"새로운 입력: {new_input}\n"
            f"요약된 선호도:"
        )

        return await async_invoke_llm(prompt, "사용자 선호도 요약")

    async def summarize_final_query(self, query):

        prompt = (
            f"다음 내용을 하나의 자연스러운 문장으로 정제해라:\n"
            f"{query}\n"
            f"정제된 검색 쿼리:"
        )

        return await async_invoke_llm(prompt, "최종 쿼리 정제")

    def robust_parse_decision_response(self, response_text):

        action_match = re.search(r"행동\s*[:：]\s*\"?([^\"\n]+)\"?", response_text)

        action = action_match.group(1).strip() if action_match else None

        book_info_match = re.search(
            r"추천\s*책\s*정보\s*[:：]\s*\"?([^\"\n]+)\"?", response_text
        )

        book_info = book_info_match.group(1).strip() if book_info_match else ""

        follow_match = re.search(
            r"추가\s*질문\s*[:：]\s*\"?([^\"\n]+)\"?", response_text
        )

        additional_question = follow_match.group(1).strip() if follow_match else ""

        return action, book_info, additional_question

    async def generate_answer(self, query):

        # Keyword - author 기준 필터링 : 아직 미구현

        author_match = re.search(r"(?:저자|작가)\s*[:：]\s*(\S+)", query)

        if author_match:

            author_name = author_match.group(1).strip().lower()

            dense_results = self.qa_chain.invoke(query)["source_documents"]

            keyword_results = [
                doc
                for doc in self.documents
                if doc.metadata.get("author", "").strip().lower() == author_name
            ]

            source_docs = list(
                {
                    doc.metadata.get("ISBN"): doc
                    for doc in (dense_results + keyword_results)
                    if doc.metadata.get("ISBN")
                }.values()
            )

        else:

            result = self.qa_chain.invoke(query)

            source_docs = result["source_documents"]

        retrieved_isbns = set()

        for doc in source_docs:

            isbn = doc.metadata.get("ISBN")

            if isbn:

                retrieved_isbns.add(isbn)

        aggregated_docs = []

        for isbn in retrieved_isbns:

            book_docs = [
                doc for doc in self.documents if doc.metadata.get("ISBN") == isbn
            ]

            if not book_docs:
                continue

            aggregated_text = "\n".join([doc.page_content for doc in book_docs])

            aggregated_docs.append(
                Document(page_content=aggregated_text, metadata=book_docs[0].metadata)
            )

        formatted_answers = []

        for doc in aggregated_docs:

            metadata = doc.metadata

            title = metadata.get("title") or extract_field(doc.page_content, "제목")

            author = metadata.get("author") or extract_field(doc.page_content, "저자")

            aggregated_text = doc.page_content

            book_intro = extract_field(aggregated_text, "책소개")

            publisher_review = extract_field(aggregated_text, "출판사리뷰")

            recommendation_field = extract_field(aggregated_text, "추천사")

            if book_intro and len(book_intro.strip()) >= MIN_INFO_LENGTH:

                selected_info = book_intro

            elif publisher_review and len(publisher_review.strip()) >= MIN_INFO_LENGTH:

                selected_info = publisher_review
            elif (
                recommendation_field
                and len(recommendation_field.strip()) >= MIN_INFO_LENGTH
            ):

                selected_info = recommendation_field

            else:

                selected_info = ""

            if not selected_info:

                reason = "추천 정보 생성 불가"

            else:

                reason_prompt = (
                    f"다음 정보와 '최종 검색 쿼리'를 참고하여, 이 책이 추천되는 이유를 명확하게 요약해라. "
                    f"책의 주요 특징과 강점을 중심으로 설명해라.\n\n정보:\n{selected_info}"
                )

                generated_reason = await async_invoke_llm(
                    reason_prompt, "추천 이유 생성"
                )
                if (
                    not generated_reason
                    or len(generated_reason) < 10
                    or "추천 정보 생성 불가" in generated_reason
                ):

                    reason = "추천 정보 생성 불가"

                else:

                    reason = generated_reason

            formatted = f"책 제목: {title}\n저자: {author}\n추천 이유: {reason}"

            formatted_answers.append(formatted)

        answer = "\n\n".join(formatted_answers)

        refined_answer = await async_invoke_llm(
            f"아래 원본 추천 결과를 읽고 사용자 선호도를 바탕으로, 각 책의 정보를 다음 형식에 맞춰 재작성해라.\n\n형식:\n책 제목: <책 제목>\n저자: <저자>\n추천 이유: <추천 이유>\n\n원본 추천 결과:\n{answer}\n\n출력 시, 반드시 위 형식만을 사용하고 불필요한 안내 문구는 포함하지 말아라.",
            "추천 결과 재정제",
        )

        return refined_answer, None

    def print_chat_history(self):

        print("-" * 50)

        for line in self.query_history[-4:]:

            if line.startswith("사용자:"):

                print(f"[사용자] {line.split(':', 1)[1].strip()}")

            elif line.startswith("챗봇:"):

                print(f"[챗봇] {line.split(':', 1)[1].strip()}")

        print("-" * 50)

    async def search_and_generate_answer(self, user_query):

        self.preferences_text = " ".join(self.user_preferences["preferences"])

        query_summary = "\n".join(self.query_history[-5:])

        if self.config.get("persona") == "Literature":

            persona_info = "감성, 현재 기분, 선호하는 문학 장르 및 작가 정보"

        elif self.config.get("persona") == "Science":

            persona_info = "초심자 여부, 관심 분야, 구체적인 기술 정보"

        else:

            persona_info = ""

        followup_count = 0

        max_followup = 2

        previous_questions = []

        while True:

            prompt_vars = {
                "history": query_summary,
                "query": user_query,
                "role_instructions": self.config["role_instructions"],
            }

            decision_result = await async_invoke(
                self.decision_chain, prompt_vars, "행동 결정"
            )

            decision_text = decision_result.get("text", "").strip()

            print(f"\n[디버그] Decision 응답: {decision_text}")

            action, book_info, additional_question = (
                self.robust_parse_decision_response(decision_text)
            )

            print(f"[디버그] 행동: {action}")

            # 중복 질문 방지 + 최대 질문 횟수 제한
            if action == "추가 질문":

                if followup_count >= max_followup:

                    print("[최대 보충 질문 수 초과 → 추천으로 전환합니다]")

                    action = "추천"

                else:

                    # 유사 질문 판단 (추가 질문일 경우)
                    new_emb = np.array(
                        self.embedding_model.embed_query(additional_question)
                    ).reshape(1, -1)

                    prev_embs = [
                        np.array(self.embedding_model.embed_query(q)).reshape(1, -1)
                        for q in previous_questions
                    ]

                    # 로그 추가: 각 유사도 확인
                    for i, emb in enumerate(prev_embs):

                        similarity = cosine_similarity(new_emb, emb)[0][0]

                        print(
                            f"[유사도 체크] '{additional_question}' vs 이전 질문[{i}] '{previous_questions[i]}' → 유사도: {similarity:.4f}"
                        )

                    # 판단 조건
                    if any(
                        cosine_similarity(new_emb, emb)[0][0] > 0.8 for emb in prev_embs
                    ):

                        print("[유사한 질문 감지 → 추천으로 전환합니다]")

                        action = "추천"

                    else:
                        previous_questions.append(additional_question)

                        self.query_history.append(f"챗봇: {additional_question}")

                        self.print_chat_history()

                        print(f"[챗봇] {additional_question}")

                        raw_user_input = input("[사용자] ")

                        self.query_history.append(f"사용자: {raw_user_input}")

                        followup_count += 1

                        if self.preferences_text:
                            updated_pref = await self.summarize_user_preferences(
                                self.preferences_text, raw_user_input
                            )
                        else:
                            updated_pref = raw_user_input

                        self.preferences_text = updated_pref

                        print(
                            f"\n[디버그] 업데이트된 사용자 선호도: {self.preferences_text}"
                        )

                        if len(self.preferences_text.split()) >= 10:

                            print(
                                "[디버그] 충분한 선호도가 수집되어 추천으로 전환합니다."
                            )

                            action = "추천"

                        else:

                            continue

            if action == "추천":

                final_query_vars = {
                    "history": "\n".join(self.query_history[-5:]),
                    "query": user_query,
                    "persona_info": persona_info,
                    "preferences": self.preferences_text,
                }

                final_query_result = await async_invoke(
                    self.final_query_generation_chain,
                    final_query_vars,
                    "최종 쿼리 생성",
                )

                final_response_text = final_query_result.get("text", "").strip()

                if final_response_text.startswith("추가 질문:"):

                    print("[논리 오류] 추천 결정 후 추가 질문이 생성됨 → fallback 적용")

                    final_query = await self.summarize_final_query(
                        f"{self.preferences_text} 기반으로 책 추천 받고 싶어"
                    )
                elif final_response_text.startswith("쿼리:"):

                    final_query = final_response_text[len("쿼리:") :].strip()

                else:

                    final_query = await self.summarize_final_query(final_response_text)

                print(f"\n[디버그] 최종 검색 쿼리: {final_query}")

                answer, _ = await self.generate_answer(final_query)

                return answer

            # fallback
            print("[fallback] 예상치 못한 흐름입니다. 기본 추천 쿼리로 전환합니다]")

            fallback_query = await self.summarize_final_query(
                f"{self.preferences_text} 기반으로 책 추천 받고 싶어"
            )

            answer, _ = await self.generate_answer(fallback_query)

            return answer

    async def interactive_multi_turn_qa(self):

        self.print_chat_history()

        while True:

            user_query = input("[사용자] ")

            if user_query.lower() == "quit":

                print("\n[대화 저장 중...]")

                print("대화 저장 완료")

                import sys

                sys.exit()

            self.query_history.append(f"사용자: {user_query}")

            answer = await self.search_and_generate_answer(user_query)

            if answer is not None:

                self.query_history.append(f"챗봇: {answer}")

                print("-" * 50)

                print(f"[사용자] {user_query}")

                print(f"[챗봇] {answer}")

                print("-" * 50)

In [ ]:
# 페르소나별 파이프라인 클래스


class LiteratureRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, retriever, qa_chain, documents, embedding_model):
        config = {"persona": "Literature", "role_instructions": literature_role}
        super().__init__(config, llm, retriever, qa_chain, documents, embedding_model)


class ScienceRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, retriever, qa_chain, documents, embedding_model):
        config = {"persona": "Science", "role_instructions": science_role}
        super().__init__(config, llm, retriever, qa_chain, documents, embedding_model)


class GeneralRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, retriever, qa_chain, documents, embedding_model):
        config = {"persona": "General", "role_instructions": general_role}
        super().__init__(config, llm, retriever, qa_chain, documents, embedding_model)

In [ ]:
# 추가 비동기 작업 허용
nest_asyncio.apply()


# 메인 함수 -비동기 실행
def main():
    print(
        "안녕하세요! 저는 도서 추천 챗봇입니다. 아래 관심 분야 중 하나를 선택해주세요."
    )
    # print("페르소나 선택:")
    print("1. 예술/문학")
    print("2. 과학/기술")
    print("3. 기본/범용")
    choice = input("원하는 페르소나 번호를 입력하세요 (1 또는 2 또는 3): ").strip()
    if choice == "1":
        pipeline = LiteratureRAGPipeline(
            llm_clova,
            dense_retriever,
            dpr_qa_chain,
            documents,
            embedding_model=ncp_embeddings,
        )
        greeting = "안녕하세요! 감성적이고 문학적인 도서 추천 챗봇입니다. 어떤 책을 읽고 싶으신가요?"
    elif choice == "2":
        pipeline = ScienceRAGPipeline(
            llm_clova,
            dense_retriever,
            dpr_qa_chain,
            documents,
            embedding_model=ncp_embeddings,
        )
        greeting = "안녕하십니까. 정확하고 논리적인 과학/기술 도서 추천 챗봇입니다. 관심 있는 분야에 대해 편하게 이야기해 주세요."
    elif choice == "3":
        pipeline = GeneralRAGPipeline(
            llm_clova,
            dense_retriever,
            dpr_qa_chain,
            documents,
            embedding_model=ncp_embeddings,
        )
        greeting = "안녕하세요! 친절하고 신뢰할 수 있는 범용 도서 추천 챗봇입니다. 어떤 책을 찾으시나요?"
    else:
        print("잘못된 선택입니다. 기본/범용 페르소나로 실행합니다.")
        pipeline = GeneralRAGPipeline(
            llm_clova,
            dense_retriever,
            dpr_qa_chain,
            documents,
            embedding_model=ncp_embeddings,
        )
        greeting = "안녕하세요! 친절하고 신뢰할 수 있는 범용 도서 추천 챗봇입니다. 어떤 책을 찾으시나요?"

    pipeline.query_history.append(f"챗봇: {greeting}")

    # nest_asyncio으로 비동기 작업을 추가?해야함 - asyncio.run() or get_event_loop().run_until_complete() 모두 사용 가능
    asyncio.run(pipeline.interactive_multi_turn_qa())


if __name__ == "__main__":
    main()

안녕하세요! 저는 도서 추천 챗봇입니다. 아래 관심 분야 중 하나를 선택해주세요.
1. 예술/문학
2. 과학/기술
3. 기본/범용
--------------------------------------------------
[챗봇] 안녕하세요! 감성적이고 문학적인 도서 추천 챗봇입니다. 어떤 책을 읽고 싶으신가요?
--------------------------------------------------

[디버그] 행동 결정 호출 전 변수: {'history': '챗봇: 안녕하세요! 감성적이고 문학적인 도서 추천 챗봇입니다. 어떤 책을 읽고 싶으신가요?\n사용자: 심심해', 'query': '심심해', 'role_instructions': '너는 감성적이고 문학적인 도서 추천 챗봇이다. 사용자의 감정과 취향을 섬세하게 파악하여, 감성적인 언어로 책을 추천해라.'}
[디버그] 행동 결정 결과: {'history': '챗봇: 안녕하세요! 감성적이고 문학적인 도서 추천 챗봇입니다. 어떤 책을 읽고 싶으신가요?\n사용자: 심심해', 'query': '심심해', 'role_instructions': '너는 감성적이고 문학적인 도서 추천 챗봇이다. 사용자의 감정과 취향을 섬세하게 파악하여, 감성적인 언어로 책을 추천해라.', 'text': '행동: 추가 질문 \n추가 질문 : 심심하실 때 주로 읽으시는 장르나 관심 있는 주제가 있으신가요?'}

[디버그] Decision 응답: 행동: 추가 질문 
추가 질문 : 심심하실 때 주로 읽으시는 장르나 관심 있는 주제가 있으신가요?
[디버그] 행동: 추가 질문
--------------------------------------------------
[챗봇] 안녕하세요! 감성적이고 문학적인 도서 추천 챗봇입니다. 어떤 책을 읽고 싶으신가요?
[사용자] 심심해
[챗봇] 심심하실 때 주로 읽으시는 장르나 관심 있는 주제가 있으신가요?
--------------------------------------------------
[챗봇]

KeyboardInterrupt: 

In [ ]:
import pandas as pd

df = pd.read_csv(
    "/Users/mj/Library/CloudStorage/OneDrive-개인/AI_study/aiffelthon_clabi/data/aiffel_book_250318_semifinal(filled_contents).csv"
)

# 제목에서 누워서 읽는 법학

In [ ]:
df[df["제목"].str.contains("토지 9")]

,Unnamed: 0,ISBN,분류,제목,부제,저자,발행자,발행일,페이지,가격,표지,책소개,저자소개,목차,출판사리뷰,추천사
13,14,9791130699554,"['국내도서 > 소설/시/희곡 > 역사소설', '국내도서 > 소설/시/희곡 > 한국...",토지 9,3부 1권,박경리 저 지음,다산책방,2023-06-06T15:00:00.000Z,516,17000,https://image.aladin.co.kr/product/31830/75/co...,54년 만에 현대적 감각으로 재탄생한 우리 시대 최고의 고전 ‘토지’! “어떠한 역...,"저 : 박경리 (Park, Kyung-Ree,朴景利,박금이) 1926년 10월 28...",제1편 만세(萬歲) 이후\n1장 끈 떨어진 연\n2장 전주행(全州行)\n3장 겨울 ...,“제 삶이 평탄했다면 글을 쓰지 않았을 것입니다. 삶이 문학보다 먼저지요.” 고전의...,NaN


In [ ]:
# query = "코딩 관련 책?"
# vectorstore.similarity_search(query, k=1)